In [1]:
!wget https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv

--2022-11-27 14:55:50--  https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/plain]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  --.-KB/s    in 0.005s  

2022-11-27 14:55:51 (94.0 MB/s) - ‘spam.csv’ saved [503663/503663]



In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 37.5 MB/s 
     |████████████████████████████████| 7.6 MB 58.3 MB/s 
     |████████████████████████████████| 182 kB 71.3 MB/s 


In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import regex as re
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModel, AutoConfig

In [4]:
data = pd.read_csv("/content/spam.csv", encoding = "ISO-8859-1")
data.drop(["Unnamed: 2", "Unnamed: 3","Unnamed: 4"], axis=1, inplace=True)
data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
data['gt'] = data['v1'].map({'ham':0,'spam':1})
data['text'] = data.v2.copy()
data['text'] = data.text.apply(lambda x: x.lower())
data.head()

,v1,v2,gt,text
0,ham,"Go until jurong point, crazy.. Available only ...",0,"go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,0,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,0,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"nah i don't think he goes to usf, he lives aro..."


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data["gt"], test_size=0.2, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [7]:
sentences = data['text']
labels = data['gt']
len(sentences), len(labels)

(5572, 5572)

# 1. Chạy mô hình bert-base-cased để xem kết quả thế nào

In [8]:
MAX_LENGTH = 128
NUM_CLASSES = len(data["gt"].unique())
MODEL_NAME = 'bert-base-cased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False, add_special_tokens=True,
                                                max_length=MAX_LENGTH, pad_to_max_length=True, use_fast=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [10]:
def Tokenizer(sentences):
    input_ids=[]
    attention_masks=[]

    for sent in tqdm(sentences):
        bert_inp=tokenizer.encode_plus(sent, add_special_tokens = True ,max_length=MAX_LENGTH, pad_to_max_length = True, return_attention_mask = True)
        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])

    input_ids=np.asarray(input_ids)
    attention_masks=np.array(attention_masks)
    return input_ids, attention_masks

In [11]:
input_ids_train, attention_masks_train = Tokenizer(X_train)
input_ids_test, attention_masks_test = Tokenizer(X_test)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 8320.48it/s]


In [12]:
def create_model(config=MODEL_CONFIG):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

    input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(20, activation='relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
    return model

In [13]:
model = create_model()
model.summary()

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [14]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              metrics=["accuracy"])

In [15]:
# mc = tf.keras.callbacks.ModelCheckpoint(filepath="./best_model.hdf5",
#                                         monitor="val_accuracy",
#                                         verbos=1,
#                                         save_best_only=True,
#                                         mode="max")

history = model.fit([input_ids_train, attention_masks_train], 
                    y_train,
                    batch_size=32, 
                    epochs=5,
                    callbacks=None,
                    validation_data=([input_ids_valid, attention_masks_vaild], y_valid)
                    )

Epoch 1/5


105/105 [==============================] - 108s 883ms/step - loss: 0.2159 - accuracy: 0.9180 - val_loss: 0.0536 - val_accuracy: 0.9848
Epoch 2/5
105/105 [==============================] - 92s 874ms/step - loss: 0.0777 - accuracy: 0.9820 - val_loss: 0.0466 - val_accuracy: 0.9874
Epoch 3/5
105/105 [==============================] - 92s 873ms/step - loss: 0.0509 - accuracy: 0.9874 - val_loss: 0.0581 - val_accuracy: 0.9865
Epoch 4/5
105/105 [==============================] - 92s 876ms/step - loss: 0.0318 - accuracy: 0.9922 - val_loss: 0.0599 - val_accuracy: 0.9857
Epoch 5/5
105/105 [==============================] - 92s 874ms/step - loss: 0.0309 - accuracy: 0.9931 - val_loss: 0.0633 - val_accuracy: 0.9874


In [16]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 10s 276ms/step - loss: 0.0335 - accuracy: 0.9955


[0.033514901995658875, 0.9955157041549683]

# 2. Thay Bert bằng xlnet-base-cased để chạy thử xem kết quả thế nào

In [17]:
MODEL_NAME = 'xlnet-base-cased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [32]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False, add_special_tokens=True,
                                                max_length=MAX_LENGTH, pad_to_max_length=True, use_fast=True)

In [19]:
input_ids_train, attention_masks_train = Tokenizer(X_train)
input_ids_test, attention_masks_test = Tokenizer(X_test)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 6966.40it/s]


In [39]:
model_2 = create_model(MODEL_CONFIG)
model_2.summary()

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tfxl_net_model_2 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_token[0][0]',            
 )                              last_hidden_state=(               'masked_token[0][0]']           
                                None, 128, 768),                                                  
                                 mems=((128, None,                                          

In [40]:
model_2.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              metrics=["accuracy"])

In [41]:
# mc = tf.keras.callbacks.ModelCheckpoint(filepath="./best_model_2.hdf5",
#                                         monitor="val_accuracy",
#                                         verbos=1,
#                                         save_best_only=True,
#                                         mode="max")

history = model_2.fit([input_ids_train, attention_masks_train], 
                    y_train,
                    batch_size=32, 
                    epochs=5,
                    callbacks=None,
                    validation_data=([input_ids_valid, attention_masks_vaild], y_valid)
                    )

Epoch 1/5


105/105 [==============================] - 135s 1s/step - loss: 0.3673 - accuracy: 0.8181 - val_loss: 0.1394 - val_accuracy: 0.9695
Epoch 2/5
105/105 [==============================] - 118s 1s/step - loss: 0.1623 - accuracy: 0.9333 - val_loss: 0.0617 - val_accuracy: 0.9821
Epoch 3/5
105/105 [==============================] - 118s 1s/step - loss: 0.1211 - accuracy: 0.9581 - val_loss: 0.0486 - val_accuracy: 0.9839
Epoch 4/5
105/105 [==============================] - 118s 1s/step - loss: 0.0937 - accuracy: 0.9692 - val_loss: 0.0573 - val_accuracy: 0.9857
Epoch 5/5
105/105 [==============================] - 118s 1s/step - loss: 0.0825 - accuracy: 0.9728 - val_loss: 0.0586 - val_accuracy: 0.9857


In [42]:
model_2.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 12s 344ms/step - loss: 0.0294 - accuracy: 0.9919


[0.02939586527645588, 0.9919282793998718]

# 3. Theo như bài tập số 2 ở những layers cuối hãy thay thành các FC layers có số units lần lượt là 50, 30, 10

In [48]:
def create_model_3(config=MODEL_CONFIG):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

    input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(50, activation='relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(30, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
    return model

In [49]:
model_3 = create_model_3()
model_3.summary()

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tfxl_net_model_4 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_token[0][0]',            
 )                              last_hidden_state=(               'masked_token[0][0]']           
                                None, 128, 768),                                                  
                                 mems=((128, None,                                          

In [50]:
model_3.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              metrics=["accuracy"])

In [51]:
# mc = tf.keras.callbacks.ModelCheckpoint(filepath="./best_model_3.hdf5",
#                                         monitor="val_accuracy",
#                                         verbos=1,
#                                         save_best_only=True,
#                                         mode="max")

history = model_3.fit([input_ids_train, attention_masks_train], 
                    y_train,
                    batch_size=32, 
                    epochs=5,
                    callbacks=None,
                    validation_data=([input_ids_valid, attention_masks_vaild], y_valid)
                    )

Epoch 1/5


105/105 [==============================] - 135s 1s/step - loss: 0.5677 - accuracy: 0.7469 - val_loss: 0.2975 - val_accuracy: 0.8735
Epoch 2/5
105/105 [==============================] - 118s 1s/step - loss: 0.2883 - accuracy: 0.8767 - val_loss: 0.1018 - val_accuracy: 0.9776
Epoch 3/5
105/105 [==============================] - 118s 1s/step - loss: 0.1675 - accuracy: 0.9402 - val_loss: 0.0587 - val_accuracy: 0.9848
Epoch 4/5
105/105 [==============================] - 118s 1s/step - loss: 0.1257 - accuracy: 0.9575 - val_loss: 0.0533 - val_accuracy: 0.9839
Epoch 5/5
105/105 [==============================] - 118s 1s/step - loss: 0.1213 - accuracy: 0.9608 - val_loss: 0.0442 - val_accuracy: 0.9874


In [52]:
model_3.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 12s 344ms/step - loss: 0.0386 - accuracy: 0.9848


[0.038551460951566696, 0.9847533702850342]